In [286]:
import requests
import urllib3.request
import time
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

# Finding Links

In [464]:
links_2019 = []

In [465]:
url = 'http://www.ufcstats.com/statistics/events/completed'
req = requests.get(url, headers)
soup = BeautifulSoup(req.content, 'html.parser')
links_arr = soup.find_all("a" , class_="b-link b-link_style_black")
    

In [466]:
for i in links_arr:
    links_2019.append(i['href'].encode('UTF8'))

In [467]:
url = 'http://www.ufcstats.com/statistics/events/completed?page=2'
req = requests.get(url, headers)
soup = BeautifulSoup(req.content, 'html.parser')
links_arr = soup.find_all("a" , class_="b-link b-link_style_black")

In [468]:
for i in links_arr:
    links_2019.append(i['href'].encode('UTF8'))

In [469]:
links_2019 = links_2019[0:-7]
# Removing Non 2019 Fight Cards

In [470]:
links_2019

['http://www.ufcstats.com/event-details/74fefd43f073cd2f',
 'http://www.ufcstats.com/event-details/4565d435005319c0',
 'http://www.ufcstats.com/event-details/b09890ba7ce1d1e2',
 'http://www.ufcstats.com/event-details/81ca2c245b19b3c5',
 'http://www.ufcstats.com/event-details/8d5daf67983b65ba',
 'http://www.ufcstats.com/event-details/fd87b1bbfcde9d5e',
 'http://www.ufcstats.com/event-details/df05aa15b2d66f57',
 'http://www.ufcstats.com/event-details/3ae10ac4df3df05c',
 'http://www.ufcstats.com/event-details/0941df56f6ac954b',
 'http://www.ufcstats.com/event-details/3cf68c1d17f66af7',
 'http://www.ufcstats.com/event-details/1bf49bf829964144',
 'http://www.ufcstats.com/event-details/94a5aaf573f780ad',
 'http://www.ufcstats.com/event-details/4834ff149dc9542a',
 'http://www.ufcstats.com/event-details/a79bfbc01b2264d6',
 'http://www.ufcstats.com/event-details/2c104b7e59a72629',
 'http://www.ufcstats.com/event-details/70167689d6a01793',
 'http://www.ufcstats.com/event-details/e5d03e4d966126bd

In [491]:
def UFC_to_df(links):
    WL = []
    Fighter = []
    Opponent = []
    Strikes = []
    TakeDowns = []
    Submission = []
    Passes = []
    Weight = []
    Method = []
    MethodDescription = []
    Round = []
    Time = []
    
    
    headers = requests.utils.default_headers()
    headers.update({ 'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'})
    temp_list = []
    for link in links:
        url = link
        req = requests.get(url, headers)
        soup = BeautifulSoup(req.content, 'html.parser')
        total = soup.find_all("p" , class_="b-fight-details__table-text")
        temp_list = []
        for i in total:
    
            item = i.text.encode('UTF8') ## Taking Text and converting from unicode to string
            temp_list.append(" ".join(item.split()))
        
        count = 0
        for i in temp_list:
            if count < 2:
                if i == 'win':
                    WL.append('Win')
                    WL.append('Loss')
                    count += 2
                else:
                    WL.append('Draw')
                    count += 1

            elif count < 4:
                Fighters.append(i)
                count += 1
            elif count < 6:
                Strikes.append(i)
                count += 1
            elif count < 8:
                TakeDowns.append(i)
                count += 1
            elif count < 10:
                Submission.append(i)
                count += 1
            elif count < 12:
                Passes.append(i)
                count += 1
            elif count < 13:
                Weight.append(i)
                Weight.append(i)
                count += 1
            elif count < 14:
                Method.append(i)
                Method.append(np.nan)
                count += 1
            elif count < 15:
                MethodDescription.append(i)
                MethodDescription.append(np.nan)
                count += 1
            elif count < 16:
                Round.append(i)
                Round.append(np.nan)
                count +=1
            elif count < 17:
                Time.append(i)
                Time.append(np.nan)
                count = 0

    results = pd.DataFrame({'WL': WL, 'Fighter': Fighter, 'Strikes': Strikes, 'Take_Downs' : TakeDowns,
                  'Sub_Attempts' : Submission,
                   'Passes' : Passes,
                   'Weight' : Weight,
                   'Method' : Method,'Finish_Description' : MethodDescription,
                
                   'Round' : Round,
                   'Time' : Time
                   
                  })
    return (results)
    

In [492]:
df = UFC_to_df(links_2019)

In [496]:
df.groupby('Finish_Description').count().Method

Finish_Description
                      286
Anaconda Choke          4
Arm Triangle            8
Armbar                  9
Elbow                   2
Elbows                  9
Flying Knee             3
Guillotine Choke       10
Heel Hook               1
Kick                   16
Kimura                  1
Knee                    5
Knees                   1
Neck Crank              1
Other                   1
Other - Choke           3
Peruvian Necktie        1
Punch                  68
Punches                40
Rear Naked Choke       35
Slam                    1
Spinning Back Fist      2
Spinning Back Kick      2
Triangle Choke          7
Name: Method, dtype: int64

In [501]:
df[df.Finish_Description == 'Other']

,Fighter,Finish_Description,Method,Passes,Round,Strikes,Sub_Attempts,Take_Downs,Time,WL,Weight
68,Bryce Mitchell,Other,SUB,5,1,6,1,1,4:20,Win,Featherweight


In [502]:
df[df.Finish_Description == 'Other - Choke']

,Fighter,Finish_Description,Method,Passes,Round,Strikes,Sub_Attempts,Take_Downs,Time,WL,Weight
256,Ovince Saint Preux,Other - Choke,SUB,2,2,14,1,1,2:46,Win,Light Heavyweight
750,Alexander Yakovlev,Other - Choke,SUB,0,2,14,1,1,3:10,Win,Lightweight
888,Ben Askren,Other - Choke,SUB,0,1,0,1,1,3:20,Win,Welterweight


In [503]:
df[df.Finish_Description == 'Neck Crank']

,Fighter,Finish_Description,Method,Passes,Round,Strikes,Sub_Attempts,Take_Downs,Time,WL,Weight
440,Claudio Silva,Neck Crank,SUB,1,1,32,1,1,2:35,Win,Welterweight


In [45]:
headers = requests.utils.default_headers()
headers.update({ 'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'})

In [28]:
URL = 'http://www.ufcstats.com/event-details/4565d435005319c0'
page = requests.get(URL)

In [427]:
url = 'http://www.ufcstats.com/event-details/4565d435005319c0'
# url = 'http://www.ufcstats.com/event-details/b09890ba7ce1d1e2'
req = requests.get(url, headers)
soup = BeautifulSoup(req.content, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html class="no-js ie8 lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!-->
<html class="no-js">
 <!--<![endif]-->
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <title>
   Stats | UFC
  </title>
  <meta content="" name="description"/>
  <meta content="" name="viewport"/>
  <link href="/blocks/main.css?ver=670617" rel="stylesheet"/>
  <script src="/js/vendor/modernizr-2.6.2.min.js">
  </script>
  <script>
   (function(i,s,o,g,r,a,m){i['GoogleAnalyticsObject']=r;i[r]=i[r]||function(){
    (i[r].q=i[r].q||[]).push(arguments)},i[r].l=1*new Date();a=s.createElement(o),
    m=s.getElementsByTagName(o)[0];a.async=1;a.src=g;m.parentNode.insertBefore(a,m)
    })(window,document,'script','//www.google-analytics.com/analytics.js','ga');

    ga('create', 'UA-2855164-1', 'au

# Start Here?

In [428]:
total = soup.find_all("p" , class_="b-fight-details__table-text")

In [429]:
temp_list = []
for i in total:
    
    item = i.text.encode('UTF8') ## Taking Text and converting from unicode to string
    temp_list.append(" ".join(item.split()))


In [323]:
# temp_list = filter(lambda a: a != 'win', temp_list)

In [430]:
temp_list

['win',
 'Kamaru Usman',
 'Colby Covington',
 '175',
 '143',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 'Welterweight',
 'KO/TKO',
 'Punch',
 '5',
 '4:10',
 'win',
 'Alexander Volkanovski',
 'Max Holloway',
 '157',
 '134',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 'Featherweight',
 'U-DEC',
 '',
 '5',
 '5:00',
 'win',
 'Amanda Nunes',
 'Germaine de Randamie',
 '79',
 '40',
 '8',
 '0',
 '2',
 '1',
 '5',
 '0',
 "Women's Bantamweight",
 'U-DEC',
 '',
 '5',
 '5:00',
 'win',
 'Marlon Moraes',
 'Jose Aldo',
 '59',
 '58',
 '1',
 '0',
 '0',
 '0',
 '1',
 '0',
 'Bantamweight',
 'S-DEC',
 '',
 '3',
 '5:00',
 'win',
 'Petr Yan',
 'Urijah Faber',
 '54',
 '18',
 '2',
 '0',
 '0',
 '0',
 '1',
 '0',
 'Bantamweight',
 'KO/TKO',
 'Kick',
 '3',
 '0:43',
 'win',
 'Geoff Neal',
 'Mike Perry',
 '15',
 '5',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 'Welterweight',
 'KO/TKO',
 'Punches',
 '1',
 '1:30',
 'win',
 'Irene Aldana',
 'Ketlen Vieira',
 '26',
 '20',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 "Women's Bantamweight",
 'KO/TK

In [431]:
WL = []
Fighters = []
Strikes = []
TakeDowns = []
Submission = []
Passes = []
Weight = []
Method = []
Round = []
Time = []

In [432]:
count = 0
for i in temp_list:
    if count < 2:
        if i == 'win':
            WL.append('Win')
            WL.append('Loss')
            count += 2
        else:
            WL.append('Draw')
            count += 1
    
    elif count < 4:
#         print(count)
        Fighters.append(i)
        count += 1
    elif count < 6:
        Strikes.append(i)
        count += 1
    elif count < 8:
        TakeDowns.append(i)
        count += 1
    elif count < 10:
        Submission.append(i)
        count += 1
    elif count < 12:
        Passes.append(i)
        count += 1
    elif count < 13:
        Weight.append(i)
        Weight.append(i)
        count += 1
    elif count < 15:
        Method.append(i)
        count += 1
    elif count < 16:
        Round.append(i)
        Round.append(np.nan)
        count +=1
    elif count < 17:
        Time.append(i)
        Time.append(np.nan)
        count = 0

In [433]:
df = pd.DataFrame({'WL': WL, 'Fighter': Fighters, 'Strikes': Strikes, 'Take Downs' : TakeDowns,
                  'Sub_Attempts' : Submission,
                   'Passes' : Passes,
                   'Weight' : Weight,
                   'Method' : Method,
                   'Round' : Round,
                   'Time' : Time
                   
                  })

In [434]:
df

,Fighter,Method,Passes,Round,Strikes,Sub_Attempts,Take Downs,Time,WL,Weight
0,Kamaru Usman,KO/TKO,0,5,175,0,0,4:10,Win,Welterweight
1,Colby Covington,Punch,0,NaN,143,0,0,NaN,Loss,Welterweight
2,Alexander Volkanovski,U-DEC,0,5,157,0,0,5:00,Win,Featherweight
3,Max Holloway,,0,NaN,134,0,0,NaN,Loss,Featherweight
4,Amanda Nunes,U-DEC,5,5,79,2,8,5:00,Win,Women's Bantamweight
5,Germaine de Randamie,,0,NaN,40,1,0,NaN,Loss,Women's Bantamweight
6,Marlon Moraes,S-DEC,1,3,59,0,1,5:00,Win,Bantamweight
7,Jose Aldo,,0,NaN,58,0,0,NaN,Loss,Bantamweight
8,Petr Yan,KO/TKO,1,3,54,0,2,0:43,Win,Bantamweight
9,Urijah Faber,Kick,0,NaN,18,0,0,NaN,Loss,Bantamweight


In [209]:
improved.remove('\nwin\n')

In [237]:
temp_list

['Kamaru Usman',
 'Colby Covington',
 '175',
 '143',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 'Welterweight',
 'KO/TKO',
 'Punch',
 '5',
 '4:10',
 'win',
 'Alexander Volkanovski',
 'Max Holloway',
 '157',
 '134',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 'Featherweight',
 'U-DEC',
 '',
 '5',
 '5:00',
 'win',
 'Amanda Nunes',
 'Germaine de Randamie',
 '79',
 '40',
 '8',
 '0',
 '2',
 '1',
 '5',
 '0',
 "Women's Bantamweight",
 'U-DEC',
 '',
 '5',
 '5:00',
 'win',
 'Marlon Moraes',
 'Jose Aldo',
 '59',
 '58',
 '1',
 '0',
 '0',
 '0',
 '1',
 '0',
 'Bantamweight',
 'S-DEC',
 '',
 '3',
 '5:00',
 'win',
 'Petr Yan',
 'Urijah Faber',
 '54',
 '18',
 '2',
 '0',
 '0',
 '0',
 '1',
 '0',
 'Bantamweight',
 'KO/TKO',
 'Kick',
 '3',
 '0:43',
 'win',
 'Geoff Neal',
 'Mike Perry',
 '15',
 '5',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 'Welterweight',
 'KO/TKO',
 'Punches',
 '1',
 '1:30',
 'win',
 'Irene Aldana',
 'Ketlen Vieira',
 '26',
 '20',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 "Women's Bantamweight",
 'KO/TKO',
 'Pu

In [30]:
soup = BeautifulSoup(page.content, 'html.parser')

In [31]:
soup

<!DOCTYPE html>\n\n<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->\n<!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8"> <![endif]-->\n<!--[if IE 8]>         <html class="no-js ie8 lt-ie9"> <![endif]-->\n<!--[if gt IE 8]><!--> <html class="no-js"> <!--<![endif]-->\n<head>\n<meta charset="unicode-escape"/>\n<meta content="IE=edge" http-equiv="X-UA-Compatible"/>\n<title>\n    Stats | UFC\n  </title>\n<meta content="" name="description"/>\n<meta content="" name="viewport"/>\n<link href="/blocks/main.css?ver=16054" rel="stylesheet"/>\n<script src="/js/vendor/modernizr-2.6.2.min.js"></script>\n<script>\n    (function(i,s,o,g,r,a,m){i['GoogleAnalyticsObject']=r;i[r]=i[r]||function(){\n    (i[r].q=i[r].q||[]).push(arguments)},i[r].l=1*new Date();a=s.createElement(o),\n    m=s.getElementsByTagName(o)[0];a.async=1;a.src=g;m.parentNode.insertBefore(a,m)\n    })(window,document,'script','//www.google-analytics.com/analytics.js','ga');\n\n    ga('create', 'UA-285

In [32]:
soup.findAll('a', class = 'b-link b-link_style_black')

SyntaxError: invalid syntax (<ipython-input-32-eff90cc9f8c0>, line 1)

In [35]:
contentTable  = soup.find('div', { "class" : "b-fight-details"}) 

In [36]:
contentTable

<div class="b-fight-details">\n<div class="b-list__info-box b-list__info-box_style_large-width">\n<ul class="b-list__box-list">\n<li class="b-list__box-list-item">\n<i class="b-list__box-item-title">\n        Date:\n      </i>\n      December 14, 2019\n    </li>\n<li class="b-list__box-list-item">\n<i class="b-list__box-item-title">\n        Location:\n      </i>\n\n      Las Vegas, Nevada, USA\n    </li>\n<li class="b-list__box-list-item">\n<i class="b-list__box-item-title">\n        Attendance:\n      </i>\n      16,811\n    </li>\n</ul>\n</div>\n<div class="b-statistics__table-preview">\n<i class="b-statistics__table-preview-item">\n    Click on a row below to see in-depth event stats.\n  </i>\n<i class="b-statistics__table-preview-item">\n    Fight, Perf, Sub, and KO of the Night Bonuses:\n    <img alt="" class="b-statistics__table-status" src="http://1e49bc5171d173577ecd-1323f4090557a33db01577564f60846c.r80.cf1.rackcdn.com/fight.png"/>\n<img alt="" class="b-statistics__table-statu

In [38]:
content = soup.find('td', { "class" : 'b-fight-details__table-col l-page_align_left'})  

In [39]:
content

<td class="b-fight-details__table-col l-page_align_left" style="width:100px">\n<p class="b-fight-details__table-text">\n<a class="b-link b-link_style_black" href="http://www.ufcstats.com/fighter-details/f1b2aa7853d1ed6e">\n              Kamaru Usman\n            </a>\n</p>\n<p class="b-fight-details__table-text">\n<a class="b-link b-link_style_black" href="http://www.ufcstats.com/fighter-details/dc9572dd6ec74859">\n              Colby Covington\n            </a>\n</p>\n</td>

In [ ]:
b-link b-link_style_black

In [43]:
names = soup.find('a', { "class" : 'b-link b-link_style_black'})  

In [44]:
names

<a class="b-link b-link_style_black" href="http://www.ufcstats.com/fighter-details/f1b2aa7853d1ed6e">\n              Kamaru Usman\n            </a>

In [33]:
results = soup.find(div='b-fight-details')

In [34]:
results

In [17]:
job_elems = soup.find_all('section', class_='b-fight-details')

In [18]:
job_elems

[]

In [21]:
url = 'http://web.mta.info/developers/turnstile.html'
response = requests.get(url)

In [24]:
soup = BeautifulSoup(response.text, 'html.parser')


In [25]:
soup

\n<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01//EN"\r\n   "http://www.w3.org/TR/html4/strict.dtd">\n\n<html lang="en">\n<head>\n<title>mta.info | Turnstile Data</title>\n<meta content="text/html; charset=unicode-escape" http-equiv="Content-Type"/>\n<meta content="width=device-width, initial-scale=1" name="viewport"/>\n<!--<meta http-equiv="X-UA-Compatible" content="IE=EmulateIE7">-->\n<meta content="IE=edge" http-equiv="X-UA-Compatible"/>\n<link href="/siteimages/favicon.ico" rel="shortcut icon"/>\n<link href="/css/base.css" rel="stylesheet" type="text/css"/>\n<link href="/css/grid.css" rel="stylesheet" type="text/css"/>\n<link href="/css/topbar.css" rel="stylesheet" type="text/css"/>\n<link href="/css/formalize.css" rel="stylesheet" type="text/css"/>\n<!-- <link rel="stylesheet" type="text/css" href="/css/jquery.datepick.css"> -->\n<!-- jQuery include should be at the top -->\n<script language="javascript" src="/js/jquery-1.4.4.min.js" type="text/javascript"></script>\n<!-- Global sit

3.8524509803921574